<a href="https://colab.research.google.com/github/kchen79427/Capstone_Bruestle_Boudia_Chen/blob/dave_branch/data_pipeline/Data_Transformation_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
import re

In [68]:
pd.set_option('display.max_columns',150)

In [73]:
X  = pd.read_parquet('/content/drive/Shareddrives/STUDENT-Capstone SS23/BTS_data/OnTime_WithWeather/OnTime_w_Weather_10_19_ml_cols.parquet')

In [10]:
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293172 entries, 0 to 293171
Data columns (total 68 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Year                      293172 non-null  object 
 1   Quarter                   293172 non-null  object 
 2   Month                     293172 non-null  object 
 3   DayofMonth                293172 non-null  object 
 4   DayOfWeek                 293172 non-null  object 
 5   Reporting_Airline         293172 non-null  object 
 6   OriginAirportSeqID        293172 non-null  object 
 7   DestAirportSeqID          293172 non-null  object 
 8   Dest                      293172 non-null  object 
 9   DestStateFips             293172 non-null  object 
 10  DestStateName             293172 non-null  object 
 11  CRSDepTime                293172 non-null  object 
 12  TaxiOut                   293172 non-null  object 
 13  WheelsOff                 293172 non-null  o

In [70]:
X.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,Reporting_Airline,OriginAirportSeqID,DestAirportSeqID,Dest,DestStateFips,DestStateName,CRSDepTime,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSElapsedTime,Distance,DistanceGroup,mfr,passengers,engine_type,no_engines,w_dir_angle,w_type,w_speed_rate,sky_c_hgt,sky_c_det,vis_dist,vis_var,tmp_air,tmp_dew,sea_lvl_p,liq_precip_qty,liq_precip_dim,liq_precip_cond,sky_cov,sky_cov_base_hgt,sky_cov_cld,sky_sum_cov,sky_sum_hgt,sky_obs_tot_cov,sky_low_cld_base_hgt,at_pres_altimeter_rate,at_pres_stn_rate,w_dir_angle_d,w_type_d,w_speed_rate_d,sky_c_hgt_d,sky_c_det_d,vis_dist_d,vis_var_d,tmp_air_d,tmp_dew_d,sea_lvl_p_d,liq_precip_qty_d,liq_precip_dim_d,liq_precip_cond_d,sky_cov_d,sky_cov_base_hgt_d,sky_cov_cld_d,sky_sum_cov_d,sky_sum_hgt_d,sky_obs_tot_cov_d,sky_low_cld_base_hgt_d,at_pres_altimeter_rate_d,at_pres_stn_rate_d,Weather_Label
0,2010,1,1,1,5,AA,1129802,1014001,ABQ,35,New Mexico,1030,15.0,1054,1121,6.0,115.0,569.0,3,MCDONNELL,172.0,Turbofan,2.0,320.0,N,26.0,22000.0,9,16000.0,N,-11.0,-39.0,10311.0,99,9999.0,9.0,00,99999.0,99,9.0,99999.0,00,99999.0,99999.0,10098.0,150.0,N,15.0,22000.0,9,16093.0,N,-61.0,-122.0,10317.0,01,0.0,9.0,99,99999.0,99,9.0,99999.0,00,99999.0,10305.0,8478.0,0
1,2010,1,1,1,5,AA,1129802,1014001,ABQ,35,New Mexico,1255,24.0,1316,1347,4.0,110.0,569.0,3,MCDONNELL,172.0,Turbofan,2.0,330.0,N,36.0,22000.0,9,14000.0,N,-22.0,-44.0,10310.0,24,10.0,9.0,00,99999.0,99,9.0,99999.0,00,99999.0,99999.0,10098.0,999.0,C,0.0,22000.0,9,16000.0,N,-56.0,-117.0,10309.0,99,9999.0,9.0,99,99999.0,99,9.0,99999.0,02,99999.0,99999.0,8472.0,0
2,2010,1,1,1,5,AA,1129802,1014001,ABQ,35,New Mexico,2225,15.0,2258,2325,9.0,105.0,569.0,3,MCDONNELL,172.0,Turbofan,2.0,160.0,N,21.0,22000.0,9,16000.0,N,94.0,-61.0,10281.0,99,9999.0,9.0,99,99999.0,99,9.0,99999.0,02,99999.0,99999.0,10071.0,10.0,N,26.0,22000.0,9,16000.0,N,50.0,-128.0,10276.0,99,9999.0,9.0,99,99999.0,99,9.0,99999.0,04,99999.0,99999.0,8449.0,0
3,2010,1,1,4,1,AA,1129802,1014001,ABQ,35,New Mexico,1410,12.0,1520,1545,4.0,110.0,569.0,3,MCDONNELL,172.0,Turbofan,2.0,340.0,N,51.0,22000.0,9,16093.0,N,-33.0,-78.0,10321.0,99,9999.0,9.0,02,914.0,99,1.0,914.0,02,99999.0,10318.0,10098.0,50.0,N,21.0,22000.0,9,16093.0,N,-39.0,-94.0,10279.0,01,0.0,9.0,99,99999.0,99,9.0,99999.0,00,99999.0,10274.0,8452.0,0
4,2010,1,1,4,1,AA,1129802,1014001,ABQ,35,New Mexico,1805,17.0,1919,1945,4.0,110.0,569.0,3,MCDONNELL,172.0,Turbofan,2.0,360.0,N,62.0,22000.0,9,16093.0,N,6.0,-72.0,10342.0,99,9999.0,9.0,02,914.0,99,1.0,914.0,02,99999.0,10339.0,10118.0,350.0,N,41.0,22000.0,9,16093.0,N,-6.0,-100.0,10299.0,01,0.0,9.0,99,99999.0,99,9.0,99999.0,00,99999.0,10291.0,8466.0,0


In [107]:
non_trans_cols = ['Year', 'Quarter', 'Month',
                  'DayofMonth', 'DayOfWeek','OriginAirportSeqID',
                  'DestAirportSeqID', 'Dest','DestStateFips',
                  'DestStateName', 'CRSDepTime', 'TaxiOut', 'WheelsOff',
                  'WheelsOn', 'TaxiIn', 'CRSElapsedTime', 'Distance',
                  'DistanceGroup','passengers']

def combine_mfr(df):
  # This function maps to a consolidated list of manufacturers return a new DF
  X['mfr'] = df['mfr'].map({'MCDONNELL DOUGLAS':'MCDONNELL',
                            'BOEING': 'BOEING',
                            'MCDONNELL DOUGLAS AIRCRAFT CO': 'MCDONNELL',
                            'AIRBUS': 'AIRBUS','AIRBUS INDUSTRIE': 'AIRBUS',
                            'MCDONNELL DOUGLAS CORPORATION': 'MCDONNELL'
                            })
  return X

def clean_column_names(df):
  ins_inc = 0
  new_cols = []
  for col in df.columns:
    new_col = re.sub('Ordinal__','',col)
    new_col = re.sub('feat__','',new_col)
    df.rename({col: new_col},inplace=True,axis=1)
    if new_col in non_trans_cols:
      new_cols.insert(ins_inc, new_col)
      ins_inc += 1
    else:
      new_cols.append(new_col)
  df = df[new_cols]
  return df


# In order to leverage this for fit/transform we need to wrap into a Function Transformer
mfr_consolidate = (FunctionTransformer(combine_mfr)
                  .set_output(transform='pandas'))

column_cleaner = (FunctionTransformer(clean_column_names)
                  .set_output(transform='pandas'))


# Destinations are encoded according to number
dest_tfr = ColumnTransformer(transformers=[('dests',
                                            OrdinalEncoder(),
                                             ['Dest']
                                            )],
                             remainder='drop',
                             verbose_feature_names_out=False)

#
mfr_trf = ColumnTransformer(transformers=[('mfr',
                                           Pipeline([('mfr_consolidate', mfr_consolidate),
                                                     ('mfr_ohe',OneHotEncoder(sparse_output=False)
                                                     )]),
                                                    ['mfr']
                                                    )],
                             remainder='drop',
                             verbose_feature_names_out=False).set_output(transform='pandas')


# each column listed is passed to OneHotEncoding to be split into several cols
cols_tfr = ColumnTransformer([('columns_encoded',
                              OneHotEncoder(sparse_output=False),
                              ['Reporting_Airline',
                                'mfr',
                              'engine_type',
                                'w_type',
                                'sky_c_det',
                                'sky_cov',
                                'w_type_d',
                                'sky_c_det_d',
                                'sky_cov_d',
                                'sky_obs_tot_cov_d']
                                )],
                remainder='passthrough',
                verbose_feature_names_out=False).set_output(transform='pandas')


# The column transformers need to be joined together to recreate the dataframe
feat_union = FeatureUnion([
                           ('Ordinal',dest_tfr),
                           ('feat', cols_tfr)
                           ]).set_output(transform='pandas')


# the pipeline to complete above processing
pipe = Pipeline([
                 ('mfr_consolidate',mfr_consolidate),
                 ('feature_union', feat_union),
                 ('col_cleaner',column_cleaner)
                ])


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_function_transformer.py:343: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


In [108]:
new_X = pipe.fit_transform(X)

In [110]:
new_X.sample()

,Dest,Dest,Year,Quarter,Month,DayofMonth,DayOfWeek,OriginAirportSeqID,DestAirportSeqID,Dest,Dest,DestStateFips,DestStateName,CRSDepTime,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSElapsedTime,Distance,DistanceGroup,passengers,Reporting_Airline_AA,Reporting_Airline_AS,Reporting_Airline_B6,Reporting_Airline_CO,Reporting_Airline_DL,Reporting_Airline_F9,Reporting_Airline_FL,Reporting_Airline_MQ,Reporting_Airline_NK,Reporting_Airline_UA,Reporting_Airline_US,Reporting_Airline_VX,mfr_AIRBUS,mfr_BOEING,mfr_nan,engine_type_Reciprocating,engine_type_Turbofan,engine_type_Turbojet,w_type_9,w_type_C,w_type_N,w_type_V,sky_c_det_9,sky_c_det_C,sky_c_det_M,sky_c_det_W,sky_cov_00,sky_cov_02,sky_cov_04,sky_cov_07,sky_cov_08,sky_cov_09,sky_cov_10,sky_cov_99,w_type_d_9,w_type_d_C,w_type_d_N,w_type_d_V,sky_c_det_d_9,sky_c_det_d_C,sky_c_det_d_M,sky_c_det_d_W,sky_cov_d_00,sky_cov_d_02,sky_cov_d_04,sky_cov_d_07,sky_cov_d_08,sky_cov_d_09,sky_cov_d_10,sky_cov_d_99,sky_obs_tot_cov_d_00,sky_obs_tot_cov_d_01,sky_obs_tot_cov_d_02,sky_obs_tot_cov_d_03,sky_obs_tot_cov_d_04,sky_obs_tot_cov_d_06,sky_obs_tot_cov_d_07,sky_obs_tot_cov_d_08,sky_obs_tot_cov_d_09,sky_obs_tot_cov_d_99,no_engines,w_dir_angle,w_speed_rate,sky_c_hgt,vis_dist,vis_var,tmp_air,tmp_dew,sea_lvl_p,liq_precip_qty,liq_precip_dim,liq_precip_cond,sky_cov_base_hgt,sky_cov_cld,sky_sum_cov,sky_sum_hgt,sky_obs_tot_cov,sky_low_cld_base_hgt,at_pres_altimeter_rate,at_pres_stn_rate,w_dir_angle_d,w_speed_rate_d,sky_c_hgt_d,vis_dist_d,vis_var_d,tmp_air_d,tmp_dew_d,sea_lvl_p_d,liq_precip_qty_d,liq_precip_dim_d,liq_precip_cond_d,sky_cov_base_hgt_d,sky_cov_cld_d,sky_sum_cov_d,sky_sum_hgt_d,sky_low_cld_base_hgt_d,at_pres_altimeter_rate_d,at_pres_stn_rate_d,Weather_Label
187960,32.0,ELP,2013,4,11,13,3,1129803,1154003,32.0,ELP,48,Texas,2220,10.0,2224,2240,5.0,95.0,551.0,3,172.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,999.0,0.0,99999.0,16000.0,9,106.0,-67.0,10328.0,99,9999.0,9.0,99999.0,99,9.0,99999.0,00,99999.0,99999.0,10117.0,100.0,31.0,99999.0,16000.0,9,150.0,33.0,10226.0,99,9999.0,9.0,99999.0,99,9.0,99999.0,99999.0,99999.0,8886.0,0


In [92]:
for col in cols:
  try:
    new_X[col].value_counts()
  except Exception as e:
    print(col + ' was a problem')

Destwas a problem
Yearwas a problem
Quarterwas a problem
Monthwas a problem
DayofMonthwas a problem
DayOfWeekwas a problem
OriginAirportSeqIDwas a problem
DestAirportSeqIDwas a problem
Destwas a problem
DestStateFipswas a problem
DestStateNamewas a problem
CRSDepTimewas a problem
TaxiOutwas a problem
WheelsOffwas a problem
WheelsOnwas a problem
TaxiInwas a problem
CRSElapsedTimewas a problem
Distancewas a problem
DistanceGroupwas a problem
passengerswas a problem
Reporting_Airline_AAwas a problem
Reporting_Airline_ASwas a problem
Reporting_Airline_B6was a problem
Reporting_Airline_COwas a problem
Reporting_Airline_DLwas a problem
Reporting_Airline_F9was a problem
Reporting_Airline_FLwas a problem
Reporting_Airline_MQwas a problem
Reporting_Airline_NKwas a problem
Reporting_Airline_UAwas a problem
Reporting_Airline_USwas a problem
Reporting_Airline_VXwas a problem
mfr_AIRBUSwas a problem
mfr_BOEINGwas a problem
mfr_nanwas a problem
engine_type_Reciprocatingwas a problem
engine_type_Turb

In [93]:
new_X.columns

Index(['Ordinal__Dest', 'feat__Reporting_Airline_AA',
       'feat__Reporting_Airline_AS', 'feat__Reporting_Airline_B6',
       'feat__Reporting_Airline_CO', 'feat__Reporting_Airline_DL',
       'feat__Reporting_Airline_F9', 'feat__Reporting_Airline_FL',
       'feat__Reporting_Airline_MQ', 'feat__Reporting_Airline_NK',
       ...
       'feat__liq_precip_dim_d', 'feat__liq_precip_cond_d',
       'feat__sky_cov_base_hgt_d', 'feat__sky_cov_cld_d',
       'feat__sky_sum_cov_d', 'feat__sky_sum_hgt_d',
       'feat__sky_low_cld_base_hgt_d', 'feat__at_pres_altimeter_rate_d',
       'feat__at_pres_stn_rate_d', 'feat__Weather_Label'],
      dtype='object', length=119)

In [69]:
new_X

,Year,Quarter,Month,DayofMonth,DayOfWeek,OriginAirportSeqID,DestAirportSeqID,Dest,DestStateFips,DestStateName,CRSDepTime,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSElapsedTime,Distance,DistanceGroup,Ordinal__Dest,Reporting_Airline_AA,Reporting_Airline_AS,Reporting_Airline_B6,Reporting_Airline_CO,Reporting_Airline_DL,Reporting_Airline_F9,Reporting_Airline_FL,Reporting_Airline_MQ,Reporting_Airline_NK,Reporting_Airline_UA,Reporting_Airline_US,Reporting_Airline_VX,mfr_AIRBUS,mfr_BOEING,mfr_MCDONNELL,mfr_nan,engine_type_Reciprocating,engine_type_Turbofan,engine_type_Turbojet,w_type_9,w_type_C,w_type_N,w_type_V,sky_c_det_9,sky_c_det_C,sky_c_det_M,sky_c_det_W,sky_cov_00,sky_cov_02,sky_cov_04,sky_cov_07,sky_cov_08,sky_cov_09,sky_cov_10,sky_cov_99,w_type_d_9,w_type_d_C,w_type_d_N,w_type_d_V,sky_c_det_d_9,sky_c_det_d_C,sky_c_det_d_M,sky_c_det_d_W,sky_cov_d_00,sky_cov_d_02,sky_cov_d_04,sky_cov_d_07,sky_cov_d_08,sky_cov_d_09,sky_cov_d_10,sky_cov_d_99,sky_obs_tot_cov_d_00,sky_obs_tot_cov_d_01,sky_obs_tot_cov_d_02,sky_obs_tot_cov_d_03,sky_obs_tot_cov_d_04,sky_obs_tot_cov_d_06,sky_obs_tot_cov_d_07,sky_obs_tot_cov_d_08,sky_obs_tot_cov_d_09,sky_obs_tot_cov_d_99,passengers,no_engines,w_dir_angle,w_speed_rate,sky_c_hgt,vis_dist,vis_var,tmp_air,tmp_dew,sea_lvl_p,liq_precip_qty,liq_precip_dim,liq_precip_cond,sky_cov_base_hgt,sky_cov_cld,sky_sum_cov,sky_sum_hgt,sky_obs_tot_cov,sky_low_cld_base_hgt,at_pres_altimeter_rate,at_pres_stn_rate,w_dir_angle_d,w_speed_rate_d,sky_c_hgt_d,vis_dist_d,vis_var_d,tmp_air_d,tmp_dew_d,sea_lvl_p_d,liq_precip_qty_d,liq_precip_dim_d,liq_precip_cond_d,sky_cov_base_hgt_d,sky_cov_cld_d,sky_sum_cov_d,sky_sum_hgt_d,sky_low_cld_base_hgt_d,at_pres_altimeter_rate_d,at_pres_stn_rate_d,Weather_Label
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010,1,1,1,5,1129802,1014001,ABQ,35,New Mexico,1030,15.0,1054,1121,6.0,115.0,569.0,3,172.0,2.0,320.0,26.0,22000.0,16000.0,N,-11.0,-39.0,10311.0,99,9999.0,9.0,99999.0,99,9.0,99999.0,00,99999.0,99999.0,10098.0,150.0,15.0,22000.0,16093.0,N,-61.0,-122.0,10317.0,01,0.0,9.0,99999.0,99,9.0,99999.0,99999.0,10305.0,8478.0,0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010,1,1,1,5,1129802,1014001,ABQ,35,New Mexico,1255,24.0,1316,1347,4.0,110.0,569.0,3,172.0,2.0,330.0,36.0,22000.0,14000.0,N,-22.0,-44.0,10310.0,24,10.0,9.0,99999.0,99,9.0,99999.0,00,99999.0,99999.0,10098.0,999.0,0.0,22000.0,16000.0,N,-56.0,-117.0,10309.0,99,9999.0,9.0,99999.0,99,9.0,99999.0,99999.0,99999.0,8472.0,0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2010,1,1,1,5,1129802,1014001,ABQ,35,New Mexico,2225,15.0,2258,2325,9.0,105.0,569.0,3,172.0,2.0,160.0,21.0,22000.0,16000.0,N,94.0,-61.0,10281.0,99,9999.0,9.0,99999.0,99,9.0,99999.0,02,99999.0,99999.0,10071.0,10.0,26.0,22000.0,16000.0,N,50.0,-128.0,10276.0,99,9999.0,9.0,99999.0,99,9.0,99999.0,99999.0,99999.0,8449.0,0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010,1,1,4,1,1129802,1014001,ABQ,35,New Mexico,1410,12.0,1520,1545,4.0,110.0,569.0,3,172.0,2.0,340.0,51.0,22000.0,16093.0,N,-33.0,-78.0,10321.0,99,9999.0,9.0,914.0,99,1.0,914.0,02,99999.0,10318.0,10098.0,50.0,21.0,22000.0,16093.0,N,-39.0,-94.0,10279.0,01,0.0,9.0,99999.0,99,9.0,99999.0,99999.0,10274.0,8452.0,0
4,0.0,1.0,0.0,0.0